In [1]:
%env CUDA_VISIBLE_DEVICES=6

import math

import torch
from linear import HiggsLinear

env: CUDA_VISIBLE_DEVICES=6


In [2]:
SIZE = 8192
HADAMARD_SIZE = 1024
GRID_DIM = 1

layer = HiggsLinear(
    SIZE, SIZE,
    GRID_DIM, 256,
    device="cuda",
    dtype=torch.float16,
)

In [3]:
input = torch.rand(1, 1, SIZE, device="cuda", dtype=torch.float16)

layer(input)

tensor([[[-25.7969,  16.4375,   9.1953,  ...,  24.5312,  29.2500,  34.9688]]],
       device='cuda:0', dtype=torch.float16)

In [4]:
from aqlm.utils import unpack_int_data, _dequantize_weight

codebooks = torch.load(f"../grids/EDEN{GRID_DIM}-256.pt").half()

weight = _dequantize_weight(
    unpack_int_data(layer.codes, 8)[:,:,None],
    codebooks[None,:,None,:],
    # scales=layer.scales,
).reshape(-1, SIZE//HADAMARD_SIZE, HADAMARD_SIZE) * layer.scales[...,None]

weight = weight.reshape(-1, SIZE)

In [5]:
from fast_hadamard_transform import hadamard_transform

hadamard_transform(input.reshape(-1, HADAMARD_SIZE), scale=1/math.sqrt(HADAMARD_SIZE)).reshape(1, -1) @ weight.T

tensor([[-25.8125,  16.4375,   9.1875,  ...,  24.5312,  29.2500,  35.0000]],
       device='cuda:0', dtype=torch.float16)

In [6]:
%%time

with torch.inference_mode():
    for i in range(1000):
        layer(input)
        torch.cuda.synchronize()

CPU times: user 320 ms, sys: 1.52 ms, total: 322 ms
Wall time: 321 ms


In [7]:
%%time

with torch.inference_mode():
    for i in range(1000):
        torch.nn.functional.linear(input, weight)
        torch.cuda.synchronize()

CPU times: user 185 ms, sys: 894 µs, total: 186 ms
Wall time: 184 ms
